In [13]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp

import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.projectors_global    as proj_glob
import hylife.utilitis_FEEC.projectors_local     as proj_loc
import hylife.utilitis_FEEC.derivatives          as der
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping

In [24]:
rho_ini_phys = lambda x, y, z : np.sin(2*np.pi*x)

In [53]:
vx_ini_phys = lambda x, y, z : 1.
vy_ini_phys = lambda x, y, z : 0.
vz_ini_phys = lambda x, y, z : 0.

In [17]:
Nel       = [16, 4, 4]           # mesh generation on logical domain
bc        = [True, True, True]   # boundary conditions (True: periodic, False: else)
p         = [3, 2, 2]            # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              # number of basis functions (D functions)

args_map  = 1, [3., 2., 1]

In [18]:
proj_glob = proj_glob.projectors_3d(T, p, bc)
proj_loc  = proj_loc.projectors_local_3d(T, p, bc)

In [41]:
def rho_ini(xi1_arr, xi2_arr, xi3_arr):
    n1, n2, n3 = xi1_arr.shape
    rho = np.empty((n1, n2, n3), dtype=float)
    for i1 in range(n1):
        for i2 in range(n2):
            for i3 in range(n3):
                idx = i1, i2, i3
                xi1 = xi1_arr[idx]
                xi2 = xi2_arr[idx]
                xi3 = xi3_arr[idx]
                x = mapping.f(xi1, xi2, xi3, *args_map, 1)
                y = mapping.f(xi1, xi2, xi3, *args_map, 2)
                z = mapping.f(xi1, xi2, xi3, *args_map, 3)
                rho[idx] = rho_ini_phys(x, y, z) * mapping.det_df(xi1, xi2, xi3, *args_map)
    return rho

In [66]:
def v1_ini(xi1, xi2_arr, xi3_arr):
    n2, n3 = xi2_arr.shape
    v1 = np.empty((n2, n3), dtype=float)
    for i2 in range(n2):
        for i3 in range(n3):
            idx = i2, i3
            
            xi2 = xi2_arr[idx]
            xi3 = xi3_arr[idx]

            x = mapping.f(xi1, xi2, xi3, *args_map, 1)
            y = mapping.f(xi1, xi2, xi3, *args_map, 2)
            z = mapping.f(xi1, xi2, xi3, *args_map, 3)

            df_11 = mapping.df(xi1, xi2, xi3, *args_map, 11)
            df_21 = mapping.df(xi1, xi2, xi3, *args_map, 21)
            df_31 = mapping.df(xi1, xi2, xi3, *args_map, 31)

            v1[idx] = vx_ini_phys(x, y, z) * df_11 + vy_ini_phys(x, y, z) * df_21 + vz_ini_phys(x, y, z) * df_31
    return v1

In [ ]:
# proj_glob.assemble_V1()
# v1_coeff, v2_coeff, v3_coeff = proj_glob.PI_1([v1_ini, v2_ini, v3_ini])

In [52]:
proj_glob.assemble_V3()

In [47]:
rho_coeff = proj_glob.PI_3(rho_ini)

In [78]:
def v1_times_rho(xi1, xi2_arr, xi3_arr):
    n2, n3 = xi2_arr.shape
    fun = np.empty((n2, n3), dtype=float)
    
    xi2_arr1d = xi2_arr[:, 0]
    xi3_arr1d = xi3_arr[0, :]
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [np.array([xi1]), xi2_arr1d, xi3_arr1d], T, p, bc)
    # v1_h, v2_h, v3_h = eva.FEM_field_V1_3d([v1_coeff, v2_coeff, v3_coeff], [xi1_arr1d, xi2_arr1d, xi3_arr1d], T, p, bc)
    
    fun = rho_h.reshape(n2, n3) * v1_ini(xi1, xi2_arr, xi3_arr)
    
    return fun

In [79]:
def v2_times_rho(xi1_arr, xi2, xi3_arr):
    n1, n3 = xi1_arr.shape
    fun = np.empty((n1, n3), dtype=float)
    
    xi1_arr1d = xi1_arr[:, 0]
    xi3_arr1d = xi3_arr[0, :]
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr1d, np.array([xi2]), xi3_arr1d], T, p, bc)
    # v1_h, v2_h, v3_h = eva.FEM_field_V1_3d([v1_coeff, v2_coeff, v3_coeff], [xi1_arr1d, xi2_arr1d, xi3_arr1d], T, p, bc)
    
    fun = rho_h.reshape(n1, n3) * 0.
    
    return fun

In [80]:
def v3_times_rho(xi1_arr, xi2_arr, xi3):
    n1, n2 = xi1_arr.shape
    fun = np.empty((n1, n2), dtype=float)
    
    xi1_arr1d = xi1_arr[:, 0]
    xi2_arr1d = xi2_arr[0, :]
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr1d, xi2_arr1d, np.array([xi3])], T, p, bc)
    # v1_h, v2_h, v3_h = eva.FEM_field_V1_3d([v1_coeff, v2_coeff, v3_coeff], [xi1_arr1d, xi2_arr1d, xi3_arr1d], T, p, bc)
    
    fun = rho_h.reshape(n1, n2) * 0.
    
    return fun

In [76]:
proj_glob.assemble_V2()

In [86]:
rhov1_coeff, rhov2_coeff, rhov3_coeff = proj_glob.PI_2([v1_times_rho, v2_times_rho, v3_times_rho])

In [87]:
derivatives = der.discrete_derivatives(T, p, bc)

DIV  = derivatives.DIV_3d()

In [88]:
rho_t = -(DIV.dot(np.concatenate((rhov1_coeff.flatten(), rhov2_coeff.flatten(), rhov3_coeff.flatten())))).reshape(NbaseD[0], NbaseD[1], NbaseD[2])